# Train our Baseline Model with New Dataset (DeBERTa)

## Overview

This notebook trains a classifier using the [DeBERTa small](https://huggingface.co/microsoft/deberta-v3-small/tree/main) model, applied to a new dataset generated by the Falcon 40B model from the ER5 tweet dataset. The data is formatted as a CSV file, from our methodology from the Bootcamp day 2 reference, 04ModelTraining.ipynb.

## Key Steps and Objectives

1. **Classifier Training**: We fine-tune the DeBERTa small model on the Falcon 40B dataset to adapt it to this specific classification task.

2. **Results Visualization**: We analyze the model's performance through:
   - **Confusion Matrix**: Provides insights into accuracy and misclassification rates.
   - **ROC Curve**: Illustrates the model's performance across various thresholds.

This notebook is essential for evaluating the DeBERTa-based classifier on the new dataset and offers comprehensive insights through detailed visualizations.

In [ ]:
!jupyter kernelspec list

In [ ]:
import sys
import transformers
import datasets
import functools
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# PyTorch for deep learning
import torch
import torch.nn.functional

# Scikit-learn for data processing and metrics
import sklearn.metrics
import sklearn.model_selection
from sklearn.model_selection import train_test_split

sys.path.append(
    "/p/project/deepacf/maelstrom/haque1/AP2-Social-media-data-for-better-local-forecasts/bootcamp/AP2/helpers/plotting"
)
import test_training_distribution

sys.path.append(
    "/p/project/deepacf/maelstrom/haque1/AP2-Social-media-data-for-better-local-forecasts/bootcamp/AP2/helpers"
)
from transformer_trainer import get_trainer

In [ ]:
random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

# Load the dataset
folder_path = "/p/project/deepacf/maelstrom/haque1/dataset/"
file = "tweets_2017_01_02_03_era5_normed_no_snow.csv"
file_path = folder_path + file
# file_path = folder_path+"relivance_score_dataset_tweet_er5_01_2017.csv"
ds_tweets = pd.read_csv(file_path)

In [ ]:
# again define labels
key_tp = "content"
ds_tweets["content"] = ds_tweets[key_tp].astype(str)

In [ ]:
labels = ds_tweets["relevance"].astype(int)
# Split the indices into training and testing sets
indices_train, indices_test = train_test_split(ds_tweets.index, test_size=0.20, stratify=labels)

In [ ]:
# Load the pretrained tokenizer
model_nm = "/p/project/deepacf/maelstrom/haque1/deberta-v3-small"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_nm)
db_config_base = transformers.AutoConfig.from_pretrained(model_nm, num_labels=2)


# Function to tokenize text
def tokenize_function(examples):
    return tokenizer(examples["inputs"], padding=True, truncation=True)


# Prepare the dataset for the Hugging Face model
def get_dataset(df, tokenizer, indices_train, indices_test, train=False):
    train_df = df.loc[indices_train, ["content", "relevance"]].rename(
        columns={"content": "inputs", "relevance": "label"}
    )
    test_df = df.loc[indices_test, ["content", "relevance"]].rename(columns={"content": "inputs", "relevance": "label"})

    train_dataset = datasets.Dataset.from_pandas(train_df)
    test_dataset = datasets.Dataset.from_pandas(test_df)

    tok_train_dataset = train_dataset.map(tokenize_function, batched=True)
    tok_test_dataset = test_dataset.map(tokenize_function, batched=True)
    if train:
        return datasets.DatasetDict({"train": tok_train_dataset, "test": tok_test_dataset})
    else:
        return tok_test_dataset

In [ ]:
def tok_func(x, tokenizer):
    return tokenizer(x["inputs"], padding=True, truncation=True, max_length=512)


# Function to convert the dataset to a format used by Hugging Face
def get_dataset_from_csv(ds, tok_func, tokenizer, indices_train, indices_test, train=True):
    # df = ds[["text_normalized", "raining"]].to_pandas()
    df = ds.rename(columns={"content": "inputs", "relevance": "labels"})
    datasets_ds = datasets.Dataset.from_pandas(df)
    tok_function_partial = functools.partial(tok_func, tokenizer=tokenizer)
    tok_ds = datasets_ds.map(tok_function_partial, batched=True)
    if train:
        return datasets.DatasetDict({"train": tok_ds.select(indices_train), "test": tok_ds.select(indices_test)})
    else:
        return tok_ds

In [ ]:
# Create the dataset
dataset = get_dataset_from_csv(ds_tweets, tok_func, tokenizer, indices_train, indices_test, train=True)

In [ ]:
FOLDER_TO_OUTPUT = "./outputs"

In [ ]:
def get_model(params, db_config_base, model_nm):
    db_config = db_config_base
    if params is not None:
        db_config.update({"cls_dropout": params["cls_dropout"]})
    db_config.update({"num_labels": 2})
    model = transformers.AutoModelForSequenceClassification.from_pretrained(model_nm, config=db_config)
    return model


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    classification_report = sklearn.metrics.classification_report(
        labels, predictions, target_names=["not raining", "raining"], output_dict=True
    )
    f1_not_raining = classification_report["not raining"]["f1-score"]
    f1_raining = classification_report["raining"]["f1-score"]
    return {"f1_not_raining": f1_not_raining, "f1_raining": f1_raining}


def get_trainer(dataset, db_config_base, model_nm, FOLDER_TO_OUTPUT, parameters):
    args = transformers.TrainingArguments(
        FOLDER_TO_OUTPUT,
        learning_rate=parameters["learning_rate"],
        warmup_ratio=parameters["warmup_ratio"],
        lr_scheduler_type=parameters["lr_scheduler_type"],
        disable_tqdm=False,
        fp16=True,
        evaluation_strategy="epoch",
        per_device_train_batch_size=parameters["batch_size"],
        per_device_eval_batch_size=parameters["batch_size"],
        num_train_epochs=parameters["epochs"],
        weight_decay=parameters["weight_decay"],
        report_to="none",
        save_strategy="epoch",
        load_best_model_at_end=True,
    )
    get_model_partial = functools.partial(get_model, db_config_base=db_config_base, model_nm=model_nm)
    return transformers.Trainer(
        model_init=get_model_partial,
        args=args,
        train_dataset=dataset["train"],
        eval_dataset=dataset["test"],
        tokenizer=tokenizer,
        compute_metrics=compute_metrics,
    )


parameters = {
    "learning_rate": 8e-5,
    "batch_size": 16,
    "weight_decay": 0.01,
    "epochs": 1,
    "warmup_ratio": 0.1,
    "cls_dropout": 0.3,
    "lr_scheduler_type": "cosine",
}


db_config_base = transformers.AutoConfig.from_pretrained(model_nm)


os.makedirs(FOLDER_TO_OUTPUT, exist_ok=True)

In [ ]:
trainer = get_trainer(dataset, db_config_base, model_nm, FOLDER_TO_OUTPUT, parameters)

# Start training
trainer.train()

In [ ]:
# Create a test dataset in the format expected by Hugging Face
test_ds = get_dataset_from_csv(
    ds_tweets.loc[indices_test], tok_func, tokenizer, indices_train, indices_test, train=False
)

In [ ]:
# Select the corresponding subset of the original DataFrame for the test set
ds_test = ds_tweets.loc[indices_test]

In [ ]:
def classification_report(labels, predictions, target_names=None, output_dict=True):
    import sklearn

    if target_names is None:
        target_names = ["not raining", "raining"]
    report = sklearn.metrics.classification_report(
        labels, predictions, target_names=target_names, output_dict=output_dict
    )
    return report


def plot_roc(truth, prediction_probability, filename=None):
    import sklearn.metrics

    false_positive_rate, true_positive_rate, _ = sklearn.metrics.roc_curve(
        truth, prediction_probability, drop_intermediate=False
    )
    roc_auc = sklearn.metrics.auc(false_positive_rate, true_positive_rate)
    figure = plt.figure()
    lw = 2
    plt.plot(
        false_positive_rate,
        true_positive_rate,
        lw=lw,
        label="ROC curve (area = %.04f)" % roc_auc,
    )

    plt.plot([0, 1], [0, 1], color="navy", lw=lw, linestyle="--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver operating characteristic")
    plt.legend(loc="lower right")
    plt.show()
    if filename is not None:
        figure.savefig(filename)

In [ ]:
def check_prediction(truth, prediction, filename=None, normalize="all", output_dict=False):
    import sklearn

    report = classification_report(
        truth,
        prediction,
        target_names=["not relevance", "relevance"],
        output_dict=output_dict,
    )
    cm = sklearn.metrics.confusion_matrix(truth, prediction, normalize=normalize)
    disp = sklearn.metrics.ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=["not relevance", "relevance"])
    disp.plot()
    ax = plt.gca()
    ax.tick_params(axis="x", labelrotation=0)
    if filename is not None:
        figure = plt.gcf()
        figure.savefig(filename)
    return report

In [ ]:
sys.path.append(
    "/p/project/deepacf/maelstrom/haque1/AP2-Social-media-data-for-better-local-forecasts/bootcamp/AP2/scripts"
)
import plotting

preds = torch.nn.functional.softmax(torch.Tensor(trainer.predict(test_ds).predictions)).numpy()
prediction_probability = preds[:, 1]
predictions = preds.argmax(axis=-1)
truth = ds_test.relevance.values
classification_report(labels=truth, predictions=predictions)
plot_roc(truth=truth, prediction_probability=prediction_probability, filename=file + "_ROC.png")

In [ ]:
check_prediction(truth=truth, prediction=predictions, filename=file + "_CM.png")